# Verification and Validation (V&V)

In this notebook, we will focus on the verification and validation of the model we developed. Let's revisit the production planning model from Chapter 2. Recall that the model is defined as follows:

$$
\begin{equation}
    \begin{aligned}
        & \underset{x}{\text{maximize}}
        & & \sum_{i \in I} (r_i - c_i) \cdot x_{i} \\
        & \text{subject to}
        & & \sum_{i \in I} c_i \cdot x_i \leq b, \\
        & & & \sum_{i \in I} x_i \leq s, \\
        & & & x_i \leq p_i \quad \forall i \in I, \\
        & & & x_i \geq 0 \quad \forall i \in I.
    \end{aligned}
\end{equation}
$$

where $I$ is the set of products, $x_i$ is the production quantity of product $i$, $r_i$ is the revenue of product $i$, $c_i$ is the cost of product $i$, $b$ is the budget, $s$ is the storage capacity, and $p_i$ is the production capacity of product $i$.


##  1. Verification

Verification is the process of checking whether the model is implemented correctly. It is essentially deals with comparing what we expect the model to do with what the model actually does.


Let's start by loading the data and the parameters from the Excel file.



In [5]:
EXCEL_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQzIHhg3mZq4eGXraXQZl07kduWMhwrnUqqs_gPT6qH_V1SWI3crMZMllxG6MX1sz3QJCFBjMt9tftr/pub?output=xlsx"

In [2]:
%pip install pyomo --quiet
%pip install openpyxl --quiet
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124565 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [3]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
import matplotlib.pyplot as plt

In [6]:
df = pd.read_excel(EXCEL_URL, sheet_name='data', index_col=0).to_dict()
params = pd.read_excel(EXCEL_URL, sheet_name='params').to_dict(orient='list')
params = {name_: value_ for name_, value_ in zip(params["name"], params["val"])}

In [7]:
df

{'revenue': {1: 50, 2: 70, 3: 90},
 'cost': {1: 30, 2: 40, 3: 50},
 'production_capacity': {1: 200, 2: 150, 3: 100}}

Recall that the model is defined as follows in Pyomo:

In [9]:
#initiate the model
model = pyo.ConcreteModel()


#define the set of products
model.I = pyo.Set(initialize=[1, 2, 3])


#define the decision variables
model.x = pyo.Var(model.I, domain=pyo.Reals, initialize=0)


#define the parameters
model.r = pyo.Param(model.I, initialize=df['revenue'])
model.c = pyo.Param(model.I, initialize=df['cost'])
model.p = pyo.Param(model.I, initialize=df['production_capacity'])
model.b = pyo.Param(initialize=params['budget'])
model.s = pyo.Param(initialize=params['capacity'])

#define useful expressions
model.total_revenue = pyo.Expression(rule=lambda model: sum(model.r[i] * model.x[i] for i in model.I))
model.total_cost = pyo.Expression(rule=lambda model: sum(model.c[i] * model.x[i] for i in model.I))


#define the objective function
def f(model):
    return model.total_revenue - model.total_cost
    #return sum((model.r[i] - model.c[i]) * model.x[i] for i in model.I)
model.obj = pyo.Objective(rule=f, sense=pyo.maximize)


#define the constraints
def budget_constraint(model):
    return sum(model.c[i] * model.x[i] for i in model.I) <= model.b
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def storage_capacity(model):
    return sum(model.x[i] for i in model.I) <= model.s
model.storage_capacity = pyo.Constraint(rule=storage_capacity)

def production_capacity(model, i):
    return model.x[i] <= model.p[i]
model.production_capacity = pyo.Constraint(model.I, rule=production_capacity)

def nonnegative_domain(model, i):
    return model.x[i] >= 0
model.nonnegative_domain = pyo.Constraint(model.I, rule=nonnegative_domain)

Let's wrap the model in a function that outputs the model object with initial values using parameters passed into the function.

In [18]:
def build_production_planning_model(params, df, initial_values=0.0):
    #initiate the model
    model = pyo.ConcreteModel()


    #define the set of products
    model.I = pyo.Set(initialize=[1, 2, 3])


    #define the decision variables
    model.x = pyo.Var(model.I, domain=pyo.Reals, initialize=initial_values)


    #define the parameters
    model.r = pyo.Param(model.I, initialize=df['revenue'])
    model.c = pyo.Param(model.I, initialize=df['cost'])
    model.p = pyo.Param(model.I, initialize=df['production_capacity'])
    model.b = pyo.Param(initialize=params['budget'])
    model.s = pyo.Param(initialize=params['capacity'])

    #define useful expressions
    model.total_revenue = pyo.Expression(rule=lambda model: sum(model.r[i] * model.x[i] for i in model.I))
    model.total_cost = pyo.Expression(rule=lambda model: sum(model.c[i] * model.x[i] for i in model.I))


    #define the objective function
    def f(model):
        return model.total_revenue - model.total_cost
    model.obj = pyo.Objective(rule=f, sense=pyo.maximize)


    #define the constraints
    def budget_constraint(model):
        return sum(model.c[i] * model.x[i] for i in model.I) <= model.b
    model.budget_constraint = pyo.Constraint(rule=budget_constraint)

    def storage_capacity(model):
        return sum(model.x[i] for i in model.I) <= model.s
    model.storage_capacity = pyo.Constraint(rule=storage_capacity)

    def production_capacity(model, i):
        return model.x[i] <= model.p[i]
    model.production_capacity = pyo.Constraint(model.I, rule=production_capacity)

    def nonnegative_domain(model, i):
        return model.x[i] >= 0
    model.nonnegative_domain = pyo.Constraint(model.I, rule=nonnegative_domain)

    return model

What we did above is just writing a function that builds the model object with initial values using parameters passed into the function. With this function, we can now build the model object as follows:

In [19]:
model = build_production_planning_model(params, df, initial_values=0.0)

The objective value of the model evaluated at the initial values can be obtained as follows:

In [20]:
model.obj()

0.0

We did expect the objective value to be 0 if the initial values are 0, which is indeed the case. We can create an assert statement to check whether the objective value is 0.

In [23]:
model = build_production_planning_model(params, df, initial_values=0.0)
assert model.obj() == 0, "The objective value must be zero under the initial values of 0"

We can create another test cases by changing the initial values. For example, let's set the initial values to be 100 for all products. If we calculate manually, the objective value should be:

$$
\begin{align}
\sum_{i \in I} (r_i - c_i) \cdot x_i & = (50 - 30) \cdot 100 + (70 - 40) \cdot 100 + (90 - 50) \cdot 100 \\
& = 20 \cdot 100 + 30 \cdot 100 + 40 \cdot 100 \\
& = 2000 + 3000 + 4000 \\
& = 9000
\end{align}
$$

We can now run the model and check whether the objective value is 9000.

In [24]:
model = build_production_planning_model(params, df, initial_values=100.0)
assert model.obj() == 9000, "The objective value must be 9000 under the initial values of 100"

At this point, we have verified that the objective function appears to be correct. We can now proceed to verifying the other calculations. We have created a few expressions that we can use to verify the other calculations. For example, we can verify the total revenue and total cost.


In [25]:
model = build_production_planning_model(params, df, initial_values=0.0)
assert model.total_revenue() == 0, "The total revenue must be 0 under the initial values of 0"
assert model.total_cost() == 0, "The total cost must be 0 under the initial values of 0"

model = build_production_planning_model(params, df, initial_values=100.0)
assert model.total_revenue() == 21000, "The total revenue must be 9000 under the initial values of 100"
assert model.total_cost() == 12000, "The total cost must be 9000 under the initial values of 100"



It shall be clear at this point that using Pyomo expressions is a good practice as it helps in verifying the calculations.

It is a good practice to create a function to run verification for us. To do that, we will need both input and the expected output.

In [27]:
test_cases = [
    {
        "initial_values": 0.0,
        "expected_objective": 0.0,
        "expected_total_revenue": 0.0,
        "expected_total_cost": 0.0,
    },
    {
        "initial_values": 100.0,
        "expected_objective": 9000.0,
        "expected_total_revenue": 21000.0,
        "expected_total_cost": 12000.0,
    }
]

def verify_production_planning_model(params, df, test_cases):
    for test_case in test_cases:
        model = build_production_planning_model(params, df, initial_values=test_case["initial_values"])
        assert model.obj() == test_case["expected_objective"], "The objective value must be {} under the initial values of {}".format(test_case["expected_objective"], test_case["initial_values"])
        assert model.total_revenue() == test_case["expected_total_revenue"], "The total revenue must be {} under the initial values of {}".format(test_case["expected_total_revenue"], test_case["initial_values"])
        assert model.total_cost() == test_case["expected_total_cost"], "The total cost must be {} under the initial values of {}".format(test_case["expected_total_cost"], test_case["initial_values"])
        print(f"Test case {test_case['initial_values']} passed")

We can run the verification using these test cases:

In [28]:
verify_production_planning_model(params, df, test_cases)

Test case 0.0 passed
Test case 100.0 passed


If the model fails the assertion, we will see AssertionError raised by the system.


We can also verify whether the solution satisfies the constraints. Let's first solve the model using the solver.

In [29]:
model = build_production_planning_model(params, df, initial_values=0.0)
solver = pyo.SolverFactory('glpk', executable='/usr/bin/glpsol')
result = solver.solve(model, tee=True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmphg65xpkz.glpk.raw --wglp /tmp/tmplxbu0i5i.glpk.glp --cpxlp
 /tmp/tmpu3npsz9n.pyomo.lp
Reading problem data from '/tmp/tmpu3npsz9n.pyomo.lp'...
8 rows, 3 columns, 12 non-zeros
51 lines were read
Writing problem data to '/tmp/tmplxbu0i5i.glpk.glp'...
40 lines were written
GLPK Simplex Optimizer 5.0
8 rows, 3 columns, 12 non-zeros
Preprocessing...
2 rows, 3 columns, 6 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  5.000e+01  ratio =  5.000e+01
GM: min|aij| =  8.801e-01  max|aij| =  1.136e+00  ratio =  1.291e+00
EQ: min|aij| =  7.746e-01  max|aij| =  1.000e+00  ratio =  1.291e+00
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (3)
*     2: obj =   7.750000000e+03 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40412 bytes)
Writing basic solution to '/tmp/tmphg65xpkz.glpk.raw'...
20 l

Let's check the status of the solution.

In [30]:
#display the results
print("Status:", result.solver.status)
print("Termination condition:", result.solver.termination_condition)
print("Optimal value:", pyo.value(model.obj))
print("Optimal production quantity:")


Status: ok
Termination condition: optimal
Optimal value: 7750.0
Optimal production quantity:


Since the solver claims an optimal result, we can check if the constraints are satisfied.

In [41]:
assert pyo.value(model.budget_constraint) <= model.b, "budget used must not exceed the available budget"
assert pyo.value(model.storage_capacity) <= model.s, "the storage capacity used must not exceed the available storage capacity"
for i in model.I:
    assert pyo.value(model.production_capacity[i]) <= model.p[i], "the production capacity used must not exceed the available production capacity"
    assert pyo.value(model.nonnegative_domain[i]) >= 0, "the production quantity must be non-negative"

print("All constraints are satisfied by the solution: ", [model.x[i]() for i in model.I])

All constraints are satisfied by the solution:  [0.0, 125.0, 100.0]


## Validation

Validation is the process of checking whether the model is implemented according to user requirements and reflects the real-world problem we are trying to solve.

We can first check whether the order of magnitude of the profit, cost, and revenue looks correct by the user. Let's say, the user says it should be in the order of tens of thousands.

In [46]:
assert model.total_revenue() < 100000, "The total revenue must be less than 100000"
assert model.total_cost() < 100000, "The total cost must be less than 100000"
assert model.obj() < 100000, "The total profit must be less than 100000"

It can also be used to check whether the solution is accurate and is a realistic solution to the real-world problem we are trying to solve.

For example, let's say the user knows the current profit is only 4500, with the production quantity of 50 for all products. We can now compare if the optimal solution is close or even better than that baseline.

For example, let's say the user knows the current profit is only 4500, with the production quantity of 50 for all products. We can now compare if the optimal solution is close or even better than that baseline.

In [44]:
model.obj()

7750.0

We can write an assertion

In [47]:
assert model.obj() >= 4500, "The optimal solution must not be worse than the current solution"

Given that the obtained profit is higher, we have higher confidence now that the optimal solution indeed improves the current solution.

We could also check whether the solution is realistic and practical to use by the user. The optimal solution is:

In [45]:
for i in model.I:
    print(f"Product {i}: {pyo.value(model.x[i])}")

Product 1: 0.0
Product 2: 125.0
Product 3: 100.0


If the user agrees that these production portfolio can indeed be carried out to the production team, then we have a valid solution.


If on the other hand the user is not satisfied, perhaps due to the solution being unrealistic, then we will need to revisit the constraint and figure out what else needs to be included in the model (more constraints, updated parameters, etc).

## Conclusion

Verification & validation are essential processes in model development. They help us ensure that the model is implemented correctly and that the solution is accurate and reflects the real-world problem we are trying to solve.

In this notebook, we have demonstrated an example of how to verify and validate the production planning model. We have used Pyomo to build the model and verified the calculations using expressions. We have also created a function to run verification for us. Finally, we have validated the solution by comparing it to the user's knowledge.

There are other ways to carry out these functions. A more systematic and integrated approach is to use unit tests for verification and demo/interviews with the users for validation. This is all done to ensure the model and the resulting solution meet expectation and the objective of the project.

